In [2]:
import json
with open('./data/clixo-mapping.json', 'r') as f:
    clixo_map = json.load(f)

In [9]:
for k in clixo_map.keys():
    print(type(k))
    break

<class 'str'>


In [4]:
!wget "https://gist.githubusercontent.com/keiono/2b2e289371b7aff1f47d5e2c2a41fc2a/raw/37bb6d2bd1d81072f95721fd83d88ac3774e365c/clixo-final.cyjs"  -O ./data/clixo-final.cyjs

--2017-01-09 16:08:19--  https://gist.githubusercontent.com/keiono/2b2e289371b7aff1f47d5e2c2a41fc2a/raw/37bb6d2bd1d81072f95721fd83d88ac3774e365c/clixo-final.cyjs
Resolving gist.githubusercontent.com... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to gist.githubusercontent.com|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5726161 (5.5M) [text/plain]
Saving to: ‘./data/clixo-final.cyjs’

./data/clixo-final. 100%[===================>]   5.46M  21.2MB/s    in 0.3s    

2017-01-09 16:08:20 (21.2 MB/s) - ‘./data/clixo-final.cyjs’ saved [5726161/5726161]



In [5]:
with open('./data/clixo-final.cyjs', 'r') as f:
    clixo = json.load(f)

In [10]:
# clixo['elements']['nodes'][1:10]

In [7]:
nodes = clixo['elements']['nodes']
new_nodes = []

for n in nodes:
    d = n['data']
    clixo_id = d['id_original'].split(':')[1]
    data = {
        'id': d['id'],
        'termId': d['id_original'],
        'name': clixo_map[clixo_id]
        

    }

KeyError: '8651'